In [1]:
import numpy as np
from skimage.io import imread, imshow, imsave
import os
import matplotlib.pyplot as plt
from scipy.special import softmax
import pandas as pd
from skimage.morphology import binary_dilation, disk
from skimage.measure import regionprops, label
from seg_utils import get_mask, get_gt, create_pmask, filter_pmask, get_pseudo_ground_truths, f1_score

In [2]:
methods = ["Mesmer", "Stardist", "Cellpose", "UnMicst"]
datapath = "/home/groups/ChangLab/dataset/HMS-TMA-TNP/OHSU-TMA/Segmentations/"
sample_ids = ['Scene 002','Scene 003', 'Scene 017', 'Scene 049', 'Scene 059']

In [3]:
def get_pseudo_ground_truths(pmasks, sample_ids, methods, weights, agree_ratio):
    pseudo_gts = {}

    for s,sid in enumerate(sample_ids):
        gt = sum([pmasks[sid][method] * weights[s][m] for m,method in enumerate(methods)])
        print(np.unique(gt))
        pseudo_gts[sid] = (gt >= agree_ratio).astype('int')
    
    return pseudo_gts

# 12 Radius Equal weights

The first step here is to generate the 'probability masks' from the original segmentation masks for every sample for every method. This is done by first zeroing out the mask, then taking the point at the center of each segmented cell, setting it equal to 1, and dilating out with a disc shape and a 12-pixel radius (the radius length varies).

In [4]:
pmasks = {}

for sid in sample_ids:
    pmasks[sid] = {}
    
    for method in methods:
        method_mask = get_mask(datapath, sid, method)
        pmasks[sid][method] = create_pmask(method_mask, 12)

Next we have to generate "pseudo-ground-truth' segmentation masks by taking all locations where 3 of the 4 methods agree, using the pmasks generated in the cell above.

In [5]:
agree_ratio = 3/4 
weights = np.ones((len(sample_ids),len(methods))) * (1/len(methods)) #equal weights

In [6]:
pseudo_gts = get_pseudo_ground_truths(pmasks, sample_ids, methods, weights, agree_ratio)

[0.   0.25 0.5  0.75 1.  ]
[0.   0.25 0.5  0.75 1.  ]
[0.   0.25 0.5  0.75 1.  ]
[0.   0.25 0.5  0.75 1.  ]
[0.   0.25 0.5  0.75 1.  ]


Now we need to generate filtered probability masks where each probability mask is now filtered down to only allow one pseudo-cell-label (centroid with the dilated disc) in the probability mask to vote for only one cell label in the pseudo-ground truth mask.

In [ ]:
f_pmasks = {}

for sid in sample_ids:
    f_pmasks[sid] = {}
    
    for method in methods:
        f_pmasks[sid][method] = filter_pmask(pmasks[sid][method], label(pseudo_gts[sid]))

Now we can regenerate the Pseudo-ground-truth masks with the filtered probability masks.

In [ ]:
f_pseudo_gts = get_pseudo_ground_truths(f_pmasks, sample_ids, methods, weights, agree_ratio)

Now we can calculate the F1 score for each method against the pseudo-ground-truth for each sample

In [ ]:
f1s_per_sample = []

for sid in sample_ids:
    cum_f1s = []
    cum_ps = []
    cum_rs = []
    
    for method in methods:
        method_mask = label(f_pmasks[sid][method])
        f1 = f1_score(f_pseudo_gts[sid], method_mask)
        
        cum_f1s.append(f1)       
    f1s_per_sample.append(cum_f1s)

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))

ax.bar(np.arange(4)-0.11, np.mean(np.array(f1s_per_sample),axis=0), color='royalblue', width=0.2)

ax.set(ylim=[0.90,1])
plt.setp(ax, xticks=np.arange(4), xticklabels=methods)
plt.yticks(np.arange(0.7, 1, 0.05))
plt.ylabel('F1')
plt.show()